In [1]:
%matplotlib inline
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import datetime
import tensorflow.contrib.summary as tfsum
from tensorflow.examples.tutorials import mnist

from tqdm import tqdm

In [41]:
dataset = mnist.input_data.read_data_sets(".", reshape=False, seed=42)

tf.get_default_graph().as_default()
tf.reset_default_graph()

images = tf.placeholder(tf.float32, [None, 28, 28, 1], name="images")
labels = tf.placeholder(tf.int64, [None], name="labels")


UNITS = 784
LAYERS = 0
ACTIVATION = "relu"

activations = {
    "none": None,
    "relu": tf.nn.relu,
    "tanh": tf.nn.tanh,
    "sigmoid": tf.nn.sigmoid,
}

training_flag = tf.placeholder_with_default(False, shape=(), name="training_flag")

hidden = images

def conv_block(inputs, filters, kernel_size = (3,3), pool = False):
    conv = tf.layers.conv2d(inputs, filters, kernel_size, padding="SAME", activation=tf.nn.relu)
    res = tf.layers.batch_normalization(conv, training=training_flag)
    
    if pool:
        res = tf.layers.max_pooling2d(res, (2,2), (2,2), padding="SAME") 
    
    return res

hidden = conv_block(hidden, 32, (5,5))
hidden = conv_block(hidden, 32, (3,3), pool=True)

hidden = conv_block(hidden, 64, (5,5))
hidden = conv_block(hidden, 64, (3,3), pool=True)

hidden = conv_block(hidden, 128, (5,5))
hidden = conv_block(hidden, 128, (3,3), pool=True)


hidden = tf.layers.flatten(hidden, name="flatten")

hidden = tf.layers.dense(hidden, units=UNITS, name="hidden_a", activation=activations[ACTIVATION])
hidden = tf.layers.batch_normalization(hidden, training=training_flag)
hidden = tf.layers.dense(hidden, units=UNITS, name="hidden_b", activation=activations[ACTIVATION])

output = tf.layers.dense(hidden, 10, activation=None, name="output_layer")
predictions = tf.argmax(output, axis=1, name="prediction")

accuracy = tf.reduce_mean(tf.cast(tf.equal(labels, predictions), tf.float32))
loss = tf.losses.sparse_softmax_cross_entropy(labels, output, scope="loss")

step = tf.train.create_global_step()

update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    train = tf.train.AdamOptimizer().minimize(loss, global_step=step, name="training")

logdir = "logs/{}-{}-{}-{}".format(
        UNITS,
        LAYERS,
        ACTIVATION,
        datetime.datetime.now().strftime("%Y-%m-%d_%H%M%S"))


summary_writer = tfsum.create_file_writer(logdir, flush_millis=3000)

def layer_histogram(layer):
    with tf.variable_scope(layer, reuse=True):
        return tfsum.histogram("weights/{}".format(layer), tf.get_variable("kernel"))

summaries = {}       
with summary_writer.as_default(), tfsum.record_summaries_every_n_global_steps(30):
    weight_summaries = [
        layer_histogram("hidden_a"),
        layer_histogram("hidden_b"),
        layer_histogram("output_layer"),
    ]
    
    
    for var in tf.trainable_variables():
        if var.name.startswith("conv2d") and var.name.endswith("kernel:0"):
            print(var.shape)
            w,h,_,_ = var.shape        
            img = tf.reshape(var[:,:,0,0], (1,w,h,1))
            weight_summaries.append(tfsum.image(var.name + "/weights", img))
        
#     g = tf.get_default_graph()
#     scope = conv.name.split("/")[0]
#     with tf.variable_scope(scope, reuse=True):
#         tfsum.image("{}/filters".format(scope), tf.get_variable("kernel:0"))
    

    summaries["train"] = [tfsum.scalar("train/loss", loss),
                          tfsum.scalar("train/accuracy", accuracy),
                          *weight_summaries]
        
with summary_writer.as_default(), tfsum.always_record_summaries():
    for run in ["dev", "test"]:
        summaries[run] = [tfsum.scalar(run + "/accuracy", accuracy)]

Extracting .\train-images-idx3-ubyte.gz
Extracting .\train-labels-idx1-ubyte.gz
Extracting .\t10k-images-idx3-ubyte.gz
Extracting .\t10k-labels-idx1-ubyte.gz
(5, 5, 1, 32)
INFO:tensorflow:Summary name conv2d/kernel:0/weights is illegal; using conv2d/kernel_0/weights instead.
(3, 3, 32, 32)
INFO:tensorflow:Summary name conv2d_1/kernel:0/weights is illegal; using conv2d_1/kernel_0/weights instead.
(5, 5, 32, 64)
INFO:tensorflow:Summary name conv2d_2/kernel:0/weights is illegal; using conv2d_2/kernel_0/weights instead.
(3, 3, 64, 64)
INFO:tensorflow:Summary name conv2d_3/kernel:0/weights is illegal; using conv2d_3/kernel_0/weights instead.
(5, 5, 64, 128)
INFO:tensorflow:Summary name conv2d_4/kernel:0/weights is illegal; using conv2d_4/kernel_0/weights instead.
(3, 3, 128, 128)
INFO:tensorflow:Summary name conv2d_5/kernel:0/weights is illegal; using conv2d_5/kernel_0/weights instead.


In [42]:
EPOCHS = 10
BATCH_SIZE = 256

session = tf.Session()

session.run(tf.global_variables_initializer())

with summary_writer.as_default():
    tf.contrib.summary.initialize(session=session, graph=tf.get_default_graph())

for i in tqdm(range(EPOCHS)):
    while dataset.train.epochs_completed == i:
        X, y = dataset.train.next_batch(BATCH_SIZE)
        session.run([train, summaries["train"]], {images: X, labels: y, training_flag: True})
    session.run(summaries["dev"], {images: dataset.validation.images, labels: dataset.validation.labels, training_flag: False})
    session.run(summaries["test"], {images: dataset.test.images, labels: dataset.test.labels, training_flag: False})

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [01:09<00:00,  6.97s/it]


In [9]:
ten = tf.layers.conv2d(tf.placeholder(tf.float32, shape=[None, 24, 24, 3]), 1, (3,3))

In [ ]:
ten.name.split("/")[0]

In [14]:
ten.op.inputs[0]

<tf.Tensor 'conv2d_3/Conv2D:0' shape=(?, 22, 22, 1) dtype=float32>

In [27]:
a = tf.trainable_variables()[0]

In [31]:
a[:,:,:,0]

<tf.Tensor 'strided_slice_1:0' shape=(5, 5, 1) dtype=float32>

In [22]:
with tf.variable_scope("conv2d", reuse=True): tf.get_variable("kernel:0")

ValueError: Variable conv2d/kernel:0 does not exist, or was not created with tf.get_variable(). Did you mean to set reuse=tf.AUTO_REUSE in VarScope?